<a href="https://colab.research.google.com/github/bmanikan/projects/blob/master/PlantPythology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list -s pathology
!kaggle datasets download -d ianmoone0617/plant-pathology-resized-512-256 
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: plant_pathology_small_256/Train_1417.jpg  
  inflating: plant_pathology_small_256/Train_1418.jpg  
  inflating: plant_pathology_small_256/Train_1419.jpg  
  inflating: plant_pathology_small_256/Train_142.jpg  
  inflating: plant_pathology_small_256/Train_1420.jpg  
  inflating: plant_pathology_small_256/Train_1421.jpg  
  inflating: plant_pathology_small_256/Train_1422.jpg  
  inflating: plant_pathology_small_256/Train_1423.jpg  
  inflating: plant_pathology_small_256/Train_1424.jpg  
  inflating: plant_pathology_small_256/Train_1425.jpg  
  inflating: plant_pathology_small_256/Train_1426.jpg  
  inflating: plant_pathology_small_256/Train_1427.jpg  
  inflating: plant_pathology_small_256/Train_1428.jpg  
  inflating: plant_pathology_small_256/Train_1429.jpg  
  inflating: plant_pathology_small_256/Train_143.jpg  
  inflating: plant_pathology_small_256/Train_1430.jpg  
  inflating: plant_pathology_small_256/Train_1431.jpg  

In [ ]:
#@title Import dependencies
import os
import shutil
import glob
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random



import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Set seed value
seed_value = 43

os.environ['PYTHONHASHSEED'] = str(seed_value)

random.seed(seed_value)

#numpy seed
np.random.seed(seed_value)

#Tf seed
tf.random.set_seed(seed_value)

#Configure new global tensorflow session
from tensorflow.compat.v1.keras import backend as k
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads = 1
)

sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
k.set_session(sess)


In [ ]:
base_dir = os.path.join(os.getcwd(),'plant_pathology_small_512')
images = glob.glob(base_dir + '/*.jpg')
print(len(images))
df = pd.read_csv('train.csv')
df.shape

In [ ]:
'''
classes = ['healthy', 'multiple_diseases', 'rust', 'scab']

#Creating directories
for cl in classes:
    image_list = [i for i in df[df[cl] == 1]['image_id'] if 'Test_' not in i]
    if not os.path.exists(os.path.join(base_dir, cl)):
        os.makedirs(os.path.join(base_dir, cl))
    for i in range(len(image_list)):
        image = os.path.join(base_dir, image_list[i] + '.jpg')
        shutil.move(image, os.path.join(base_dir, cl))

#Creating  Test dataset
os.makedirs(os.path.join(base_dir, 'test'))
test_dir = os.path.join(base_dir, 'test')
test_dir
images = glob.glob(base_dir + '/*.jpg')
for t in images:
    shutil.move(t, test_dir)

#Creating Train & Validation dataset
for cl in classes:
    image_dir = os.path.join(base_dir, cl)
    images = glob.glob(image_dir + '/*.jpg')
    print(f'the number of images in {cl} class is {len(images)} images.')
    num_rec = round(len(images)*0.8)
    train, val = images[:num_rec], images[num_rec:]
    
    for t in train:
        if not os.path.exists(os.path.join(base_dir, 'train', cl)):
            os.makedirs(os.path.join(base_dir, 'train', cl))
        train_dir = os.path.join(base_dir, 'train', cl)
        shutil.move(t, train_dir)
        
    for t in val:
        if not os.path.exists(os.path.join(base_dir, 'val', cl)):
            os.makedirs(os.path.join(base_dir, 'val', cl))
        val_dir = os.path.join(base_dir, 'val', cl)
        shutil.move(t, val_dir)
'''

In [ ]:
#Specifying directories
#train_dir = os.path.join(base_dir, 'train')
#val_dir = os.path.join(base_dir, 'val')
#test_dir = os.path.join(base_dir, 'test')

#Parameters
batch_size = 32
IMG_SHAPE = 224
classes = ['healthy', 'multiple_diseases', 'rust', 'scab']

#Train dataset & Generator
image_gen_train = ImageDataGenerator(
    rotation_range = 90,
    width_shift_range = 0.2,
    shear_range = 0.1,
    zoom_range = 0.1,
    rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True,
    brightness_range = (0.1, 1.5),
    channel_shift_range = 0.1,
    validation_split = 0.2
)

'''
#old method of splitting files and creating Generator

train_data_gen = image_gen_train.flow_from_directory(
    batch_size = batch_size,
    directory = train_dir,
    shuffle = True,
    target_size = (IMG_SHAPE,IMG_SHAPE),
    class_mode = 'categorical'
)

#Validation dataset & Generator
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='categorical')


test_generator = image_gen_val.flow_from_dataframe(
    test,
    directory = base_dir,
    target_size = (IMG_SHAPE,IMG_SHAPE),
    x_col = 'image_id',
    y_col = None,
    class_mode = None,
    shuffle = False,
    batch_size = batch_size
)
'''
#forming train & validation set with Flow_from_dataframe
train = pd.read_csv('train.csv')
train['image_id'] = train['image_id'] + '.jpg'

train_gen = image_gen_train.flow_from_dataframe(
    train,
    directory = base_dir,
    target_size = (IMG_SHAPE, IMG_SHAPE),
    x_col = 'image_id',
    y_col = classes,
    class_mode = 'raw',
    batch_size = batch_size,
    subset = 'training'
)

val_gen = image_gen_train.flow_from_dataframe(
    train,
    directory = base_dir,
    target_size = (IMG_SHAPE, IMG_SHAPE),
    x_col = 'image_id',
    y_col = classes,
    class_mode = 'raw',
    shuffle = False,
    batch_size = batch_size,
    subset = 'validation'
)


#Forming test dataset & Generator
sub_path = os.path.join(os.getcwd(), 'sample_submission.csv')

test = pd.read_csv(sub_path)
test['image_id'] = test['image_id'] + '.jpg'

test_generator = image_gen_train.flow_from_dataframe(
    test,
    directory = base_dir,
    target_size = (IMG_SHAPE,IMG_SHAPE),
    x_col = 'image_id',
    y_col = None,
    class_mode = None,
    shuffle = False,
    batch_size = batch_size
)

In [ ]:
'''
import numpy as np
dataiter = iter(train_data_gen)
images, labels = dataiter.next()

fig = plt.figure(figsize=(40,8))

for idx in np.arange(20):
  ax = fig.add_subplot(2,20/2, idx+1, xticks=[], yticks=[])
  plt.imshow(images[idx])
  ax.set_title(classes[int(labels[idx])])
'''

In [ ]:
#@title VGG16 Model Base_model of acc 82%
vgg16 = tf.keras.applications.VGG16(weights='imagenet', include_top=False)
resnet = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, pooling = 'avg')

#for l in range(len(vgg16.layers)):
#  vgg16.layers[l].trainable = False
model = Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(224,224,3),name = 'image_input'))
model.add(vgg16)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(2048, activation='relu', name='fc2'))
model.add(Dense(4, activation='softmax', name='predictions'))
model.summary()

In [ ]:
resnet = hub.KerasLayer('https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4',
                        trainable = True)
def build_model(trained_model):
  model = Sequential()
  model.add(trained_model)
  #model.add(tf.keras.layers.GlobalAveragePooling2D())
  model.add(Dense(1024, activation='relu', name='fc1'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(Dense(512, activation='relu', name='fc2'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(Dense(256, activation='relu', name='fc3'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(Dense(4, activation='softmax', name='predictions'))
  model.build([None,IMG_SHAPE,IMG_SHAPE,3])
  return model

model = build_model(resnet)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 2048)              23564800  
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
fc2 (Dense)                  (None, 512)               524800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
fc3 (Dense)                  (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [ ]:
import keras
from keras.models import Model
model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=(IMG_SHAPE,IMG_SHAPE,3))
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
predictions = Dense(4, activation = 'softmax')(x)

model = Model(inputs = model.input, outputs = predictions)

model.summary()

94674944/94668760 [==============================] - 1s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________

In [ ]:
#@title Calculating class weights

#alternative way - to be tested
# from sklearn.utils import class_weight
# class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

count_h = np.sum(df['healthy'] == 1 )
count_m = np.sum(df['multiple_diseases'] == 1)
count_r = np.sum(df['rust'] == 1)
count_s = np.sum(df['scab'] == 1)
total = count_h + count_m + count_r +count_s
print(f'Example: \n     Total:{total} \n     Healthy:{count_h} ({100*count_h/total})\n     Multiple:{count_m} ({100*count_m/total})\n     Rust:{count_r} ({100*count_r/total})\n     Scab:{count_s} ({100*count_s/total})')

weights_for_0 = (1/count_h)*(total)/2.0
weights_for_1 = (1/count_m)*(total)/2.0
weights_for_2 = (1/count_r)*(total)/2.0
weights_for_3 = (1/count_s)*(total)/2.0

class_weight = {0: weights_for_0, 1: weights_for_1, 2: weights_for_2, 3: weights_for_3}

print('Weight for class 0: {:.2f}'.format(weights_for_0))
print('Weight for class 1: {:.2f}'.format(weights_for_1))
print('Weight for class 2: {:.2f}'.format(weights_for_2))
print('Weight for class 3: {:.2f}'.format(weights_for_3))

In [ ]:
#stop it early
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 20,
    restore_best_weights=True
)

#model checkpoint
checkpointfile = os.path.join(os.getcwd(), 'drive/My Drive/project/plantpathology/Nasnetmodel 18/08_Lr0.00001')

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpointfile,
    monitor = 'val_accuracy',
    save_best_only = True
)

#Learning rate scheduler
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.1,
    patience = 5,
    cooldown = 1,
    min_lr = 0.0000001,
    verbose=1
)



In [ ]:
#Focal loss:
def focal_loss(gamma=2., alpha=0.25):
  gamma = float(gamma)
  alpha = float(alpha)

  def focal_loss_fixed(y_true, y_pred):
    epsilon = 1.e-9
    y_true = tf.convert_to_tensor(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, tf.float32)

    model_out = tf.add(y_pred, epsilon)
    ce = tf.multiply(y_true, -tf.math.log(model_out))
    weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
    f1 = tf.multiply(alpha, tf.multiply(weight, ce))
    reduced_f1 = tf.reduce_max(f1, axis=1)
    return tf.reduce_mean(reduced_f1)
  return focal_loss_fixed

In [ ]:
epochs = 100

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer = opt,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(train_gen,
          epochs = epochs,
          validation_data = val_gen,
          callbacks = [earlystopping, checkpoint, reduce_lr])

Epoch 1/100
46/46 [==============================] - ETA: 0s - loss: 0.9471 - accuracy: 0.6321WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /content/drive/My Drive/project/plantpathology/Nasnetmodel 18/08_Lr0.00001/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/project/plantpathology/Nasnetmodel 18/08_Lr0.00001/assets


46/46 [==============================] - 57s 1s/step - loss: 0.9471 - accuracy: 0.6321 - val_loss: 302.2719 - val_accuracy: 0.3846
Epoch 2/100
46/46 [==============================] - 36s 786ms/step - loss: 0.7192 - accuracy: 0.7323 - val_loss: 24.5099 - val_accuracy: 0.2555
Epoch 3/100
46/46 [==============================] - ETA: 0s - loss: 0.5864 - accuracy: 0.7989INFO:tensorflow:Assets written to: /content/drive/My Drive/project/plantpathology/Nasnetmodel 18/08_Lr0.00001/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/project/plantpathology/Nasnetmodel 18/08_Lr0.00001/assets


46/46 [==============================] - 55s 1s/step - loss: 0.5864 - accuracy: 0.7989 - val_loss: 7.2042 - val_accuracy: 0.5412
Epoch 4/100
15/46 [========>.....................] - ETA: 19s - loss: 0.4846 - accuracy: 0.8458

KeyboardInterrupt: ignored

In [ ]:
images, labels = next(iter(train_data_gen))
class_pred = np.array(classes)

predicted_batch = model.predict(images)
predicted_batch = tf.squeeze(predicted_batch).numpy()

predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_pred[predicted_ids]

In [ ]:
#loading saved model from checkpointfile
imported = tf.keras.models.load_model(checkpointfile)
imported.layers[0].trainable = False
imported.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 4032)              84916818  
_________________________________________________________________
fc1 (Dense)                  (None, 2048)              8259584   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
fc2 (Dense)                  (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
fc3 (Dense)                  (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

In [ ]:
#Creating Submission file

sub = pd.read_csv(sub_path)
probs_nansnet = model.predict(test_generator)
sub.loc[:,'healthy':] = probs_nansnet

sub.to_csv('submission_nasnet.csv', index=False)
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.000019,0.001308,0.998599,0.000074
1,Test_1,0.000001,0.000182,0.999814,0.000003
2,Test_2,0.000017,0.000565,0.000021,0.999397
3,Test_3,0.999785,0.000038,0.000124,0.000053
4,Test_4,0.000032,0.003252,0.996638,0.000077


In [ ]:
from tqdm import tqdm

In [ ]:
#Test Time Augmentation
tta_steps = 10
predictions = []

for i in tqdm(range(tta_steps)):
  preds = model.predict_generator(test_generator_tta)
  predictions.append(preds)

pred = np.mean(predictions, axis = 0)
sub.loc[:,'healthy':] = pred

sub.to_csv('submission_nasnet.csv', index=False)
sub.head()

  0%|          | 0/10 [00:00<?, ?it/s]

Instructions for updating:
Please use Model.predict, which supports generators.


Instructions for updating:
Please use Model.predict, which supports generators.
100%|██████████| 10/10 [05:25<00:00, 32.59s/it]


,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.000391,0.241122,0.757001,0.001486
1,Test_1,0.000074,0.001715,0.998105,0.000106
2,Test_2,0.000686,0.001174,0.000065,0.998075
3,Test_3,0.999416,0.000031,0.000223,0.000330
4,Test_4,0.000356,0.001724,0.997872,0.000048


In [ ]:
!kaggle competitions submit -c plant-pathology-2020-fgvc7 -f submission_nasnet.csv -m "trial of nb"

100% 165k/165k [00:02<00:00, 61.1kB/s]
Successfully submitted to Plant Pathology 2020 - FGVC7